In [1]:
DATA_NAME = 'syntheticMRI3D-full' 
TRANSFORM = 'wavelet'
CHANNEL = 'add'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_add_wavelet_full_syntheticMRI3D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) <100:
            X[:, i] = np.zeros(bootstrap_size)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00
3,0.0,5098993.58,-230.39,12.58,-2.54,-1.93,0.23
4,0.0,-230.39,3750231.73,-15.30,8.13,2.59,0.44
5,0.0,12.58,-15.30,342948.06,-0.09,0.05,0.16
6,0.0,-2.54,8.13,-0.09,48126.19,0.06,0.07
7,0.0,-1.93,2.59,0.05,0.06,2423.24,-0.02
8,0.0,0.23,0.44,0.16,0.07,-0.02,53.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_21870/980427090.py:1: RuntimeWarning: invalid value encountered in divide
  corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))


,2,3,4,5,6,7,8
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.00000,-0.00005,0.00001,-0.00001,-0.00002,0.00001
4,NaN,-0.00005,1.00000,-0.00001,0.00002,0.00003,0.00003
5,NaN,0.00001,-0.00001,1.00000,-0.00000,0.00000,0.00004
6,NaN,-0.00001,0.00002,-0.00000,1.00000,0.00001,0.00004
7,NaN,-0.00002,0.00003,0.00000,0.00001,1.00000,-0.00006
8,NaN,0.00001,0.00003,0.00004,0.00004,-0.00006,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

327.280307161633

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.82868723e+12 1.89598374e+12 1.56934170e+10 2.89231107e+08
 6.52352748e+05 2.35105316e+02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,-0.0,0.970050,-0.242748,-0.008641,-0.001208,-0.000061,-0.000001
1,0.0,0.242521,0.969911,-0.021180,-0.002940,-0.000148,-0.000003
2,0.0,0.013446,0.018347,0.999098,-0.035802,-0.001758,-0.000038
3,-0.0,0.002365,0.003214,0.035686,0.999212,-0.016893,-0.000367
4,0.0,0.000159,0.000215,0.002356,0.016813,0.999796,-0.010981
5,-0.0,0.000005,0.000007,0.000077,0.000550,0.010973,0.999940
6,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0,
 0.7574793182890651,
 0.9816531490122257,
 0.9643144778159252,
 0.9831867847928972,
 0.9890269977295846,
 1.0]